In [1]:
import pandas as pd
import gc
import os
from tqdm import tqdm


In [2]:
# ▶ Base 경로
base_path = "마이박스 다운로드\\최종EDA_unique제거 전체\\train\\"

# ▶ 병합 대상 파일 목록
file_names = [
    "train_1_회원정보_모델링용.csv",
    "train_2_신용정보_모델링용.csv",
    "train_3_매출정보_unique2_학습용.csv",
    "train_4.청구입금정보_변동없음.csv",
    "train_5.잔액정보_unique.csv",
    "train_6.채널정보_unique.csv",
    "train_7_마케팅정보_unique.csv",
    "train_8_성과정보_변동없음_결측치처리.csv"
]
file_paths = [base_path + fname for fname in file_names]

# ▶ 데이터 타입 최적화 함수
def optimize_types(df):
    for col in df.select_dtypes(include='number').columns:
        if 'float' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif 'int' in str(df[col].dtype):
            df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include='object').columns:
        if df[col].nunique() / len(df) < 0.5:
            df[col] = df[col].astype('category')
    return df

# ▶ 저장 경로
step_path = "병합/merged_step_train.csv"
final_path = "병합/merged_final_train.csv"

# ▶ 재시작 여부 판단
if os.path.exists(step_path):
    print("✅ 이전 병합 파일을 불러와 이어서 진행합니다.")
    merged = pd.read_csv(step_path)
    start_idx = len(merged.columns) - 1  # ID 포함
else:
    print("🆕 새로운 병합을 시작합니다.")
    df = pd.read_csv(file_paths[0])
    df = optimize_types(df)
    merged = df.copy()
    merged.to_csv(step_path, index=False)
    print(f"✅ 첫 파일 저장 완료: {step_path}")
    del df
    gc.collect()
    start_idx = 1  # 첫 파일 이후부터 시작

# ▶ tqdm으로 병합 진행
for i in tqdm(range(start_idx, len(file_paths)), desc="📁 파일 병합 중", unit="file"):
    file = file_paths[i]
    print(f"\n📄 병합 중: {file_names[i]}")
    df = pd.read_csv(file)
    df = optimize_types(df)

    merged = pd.concat([merged, df.drop(columns=['ID'], errors='ignore')], axis=1)
    merged.to_csv(step_path, index=False)
    print(f"✅ 중간 저장 완료: {step_path} (열 {merged.shape[1]})")

    del df
    gc.collect()

# ▶ 중복 컬럼 제거 (ID 등)
merged = merged.loc[:, ~merged.columns.duplicated()]

# ▶ 최종 저장
merged.to_csv(final_path, index=False)
print(f"\n🎉 최종 병합 완료: {final_path} (행 {merged.shape[0]}, 열 {merged.shape[1]})")


🆕 새로운 병합을 시작합니다.
✅ 첫 파일 저장 완료: 병합/merged_step_train.csv


📁 파일 병합 중:   0%|                                                                         | 0/7 [00:00<?, ?file/s]


📄 병합 중: train_2_신용정보_모델링용.csv


📁 파일 병합 중:  14%|█████████▎                                                       | 1/7 [00:56<05:36, 56.12s/file]

✅ 중간 저장 완료: 병합/merged_step_train.csv (열 104)

📄 병합 중: train_3_매출정보_unique2_학습용.csv


...

📁 파일 병합 중:  86%|██████████████████████████████████████████████████████▊         | 6/7 [21:28<04:05, 245.06s/file]

✅ 중간 저장 완료: 병합/merged_step_train.csv (열 663)

📄 병합 중: train_8_성과정보_변동없음_결측치처리.csv


📁 파일 병합 중: 100%|████████████████████████████████████████████████████████████████| 7/7 [27:08<00:00, 232.69s/file]

✅ 중간 저장 완료: 병합/merged_step_train.csv (열 711)



🎉 최종 병합 완료: 병합/merged_final_train.csv (행 2400000, 열 704)


In [5]:
print(merged.shape)
# → (2400000, 871) 정도가 나와야 정상

(2400000, 704)


In [7]:
df1 = pd.read_csv('병합/merged_final_train.csv')

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_17404\1482768869.py:1: DtypeWarning: Columns (308) have mixed types. Specify dtype option on import or set low_memory=False.
  df1 = pd.read_csv('병합/merged_final_train.csv')


In [11]:
df_sample = pd.read_csv('병합/merged_final_train.csv', nrows=100)
print(df_sample.columns[308])

최종카드론_신청경로코드


In [2]:
df2 = pd.read_csv('병합/merged_final_train.csv', dtype={308: str})

In [2]:
# 데이터프레임을 넣고 column별 특성 및 결측값, 고유값들을 확인하는 함수를 작성해본다.
# 필수는 아니지만 전체적인 흐름을 파악하기 쉬워진다.

def resumetable(df, n):
    print(f'데이터셋 크기: {df.shape}')                                # 데이터프레임의 전체 크기(행, 열) 출력

    summary = pd.DataFrame(df.dtypes, columns=['데이터 타입'])         # 각 피처의 데이터 타입을 가져와 데이터프레임으로 생성
    summary = summary.reset_index()                                    # 인덱스를 초기화하여 컬럼으로 변환
    summary = summary.rename(columns={'index':'피처'})                 # 'index' 컬럼명을 '피처'로 변경

    summary['결측값 개수'] = df.isnull().sum().values                 # 각 피처의 결측값(null) 개수 계산
    summary['고유값 개수'] = df.nunique().values                      # 각 피처의 고유값 개수 계산
    
    for i in range(n):
        summary[f"{i+1}번째 값"] = df.iloc[i].values
    
    return summary                                                     # 요약 테이블 반환

In [17]:
with pd.option_context('display.max_rows', None):
    display(resumetable(df2, 3))

데이터셋 크기: (2400000, 704)


,피처,데이터 타입,결측값 개수,고유값 개수,1번째 값,2번째 값,3번째 값
0,기준년월,int64,0,6,201807,201807,201807
1,ID,object,0,400000,TRAIN_000000,TRAIN_000001,TRAIN_000002
2,남녀구분코드,int64,0,2,2,1,1
3,회원여부_이용가능,int64,0,2,1,1,1
4,회원여부_이용가능_CA,int64,0,2,1,1,1
5,회원여부_이용가능_카드론,int64,0,2,0,1,0
6,소지여부_신용,int64,0,2,1,1,1
7,소지카드수_유효_신용,int64,0,5,1,1,1
8,소지카드수_이용가능_신용,int64,0,4,1,1,1
9,입회일자_신용,int64,0,325,20130101,20170801,20080401


In [4]:
import pandas as pd

# Segment.csv 불러오기
segment_df = pd.read_csv('EDA/Segment/Segment_병합.csv')

# ▶ 안전 확인
# 1. 행 개수 확인
assert len(df2) == len(segment_df), f"❌ 행 개수 불일치: df2={len(df2)} vs segment={len(segment_df)}"

# 2. ID 순서가 정확히 같은지 확인
assert (df2['ID'].values == segment_df['ID'].values).all(), "❌ ID 컬럼의 순서가 일치하지 않습니다."

# ▶ Segment 컬럼만 붙이기
df2['Segment'] = segment_df['Segment'].values


In [5]:
df2.shape

(2400000, 705)

In [6]:
df2.to_csv('병합/merged_final_train.csv', index=False)

- 연체 부분 NaN 값이 있어 수정 후 재저장
- 308열 문제도 해결

In [8]:
df2['최종카드론_신청경로코드'] = df2['최종카드론_신청경로코드'].astype(str)

In [11]:
df2['연체일자_B0M'] = df2['연체일자_B0M'].fillna(-1)
df2['연체원금_최근'] = df2['연체원금_최근'].fillna(-1)
df2['연체일수_최근'] = df2['연체일수_최근'].fillna(-1)
df2['연체일수_B1M'] = df2['연체일수_B1M'].fillna(-1)
df2['연체일수_B2M'] = df2['연체일수_B2M'].fillna(-1)

In [12]:
df2.to_parquet('병합/merged_final_train.parquet', index=False)

In [13]:
df2.to_csv('병합/merged_final_train.csv', index=False)

# 혹시 모르니 하나씩 열어서 확인...

In [3]:
# ▶ Base 경로
base_path = "마이박스 다운로드/최종EDA_unique제거 전체/train/"

# ▶ 병합 대상 파일 목록
file_names = [
    "train_1_회원정보_모델링용.csv",
    "train_2_신용정보_모델링용.csv",
    "train_3_매출정보_unique2_학습용.csv",
    "train_4.청구입금정보_변동없음.csv",
    "train_5.잔액정보_unique.csv",
    "train_6.채널정보_unique.csv",
    "train_7_마케팅정보_unique.csv",
    "train_8_성과정보_변동없음_결측치처리.csv"
]

# ▶ 파일별 행/열 개수 확인
for fname in file_names:
    path = base_path + fname
    try:
        df = pd.read_csv(path)
        print(f"✅ {fname}: {df.shape[0]:,} rows × {df.shape[1]:,} columns")
    except Exception as e:
        print(f"❌ {fname}: 로딩 실패\n  └ 오류 메시지: {e}")


✅ train_1_회원정보_모델링용.csv: 2,400,000 rows × 65 columns
✅ train_2_신용정보_모델링용.csv: 2,400,000 rows × 40 columns
✅ train_3_매출정보_unique2_학습용.csv: 2,400,000 rows × 341 columns
✅ train_4.청구입금정보_변동없음.csv: 2,400,000 rows × 44 columns
✅ train_5.잔액정보_unique.csv: 2,400,000 rows × 73 columns
✅ train_6.채널정보_unique.csv: 2,400,000 rows × 66 columns
✅ train_7_마케팅정보_unique.csv: 2,400,000 rows × 40 columns
✅ train_8_성과정보_변동없음_결측치처리.csv: 2,400,000 rows × 49 columns
